In [4]:
# Här en kommentar som säger vad som behöver göras på den här sidan för att få den att fungera och räknas som klar:

# Referenser till källor är trasigt. John har hittat ett plugin till jupyter som kan fixa det här:
# https://github.com/takluyver/cite2c

# Just nu har vi referenser till avsnitt. Dessa kan göras om till länkar istället som hänvisar till andra sidor. 

# Eventuellt vill vi göra om den här sidan och arbeta på att anpassa detta projekt: 
# https://github.com/ChrisRackauckas/JupyterSite

# En introduktion till DFT

DFT står för Density Functional Theory, på svenska täthetsfunktionalteori, och används för att beräkna grundtillstånd för flerelektronsystem. 


Att räkna kvantmekaniskt på flerelektronssytem är svårt. För att göra det göra beräkningarna lättare
görs tre förenklingar i DFT:

 * **Born-Oppenheimer approximationen**: 
 
   Vågfunktionen för en atom kan med god noggrannhet separeras i två delar, en för atomkärnan och en
   för elektronerna kring kärnan 
   
$$\begin{equation*}
   \Psi_\text{atom} \approx \Psi_\text{atomkärna} \cdot \Psi_\text{elektroner}.
\end{equation*}$$
   
   Med denna uppdelning är det möjligt att lösa vågfunktionen för elektronerna oberoende av atomkärnans rörelse. 
   Detta kan motiveras med att atomkärnor har mycket större massa än elektroner och de rör sig därför
   med olika tidsskalor. I fortsättningen kommer $\Psi_\text{elektroner}$ kallas enbart $\Psi$.

 * **Ersätt elektronväxelverkan med potentialterm**:
 
   En ytterligare förenkling av problemet är att elektronerna ersätts av teoretiska icke-interagerande partiklar. Vågfunktionen $\Psi(x_1, ..., x_N)$ för de $N$ antal elektronerna kan då delas upp i $N$ antal ortogonala vågfunktioner $\psi_i(x_i)$, en för varje elektron. På detta sätt kan vågfunktionen för varje elektron lösas var för sig, oberoende av de andra elektronerna. För att uppdelningen ska uppfylla Pauliprincipen måste den anta formen

$$\begin{equation*}
    \Psi = \frac{1}{\sqrt{N \text{!}}} \left| \begin{matrix}
        \psi_1(\vec{x_1}) & \psi_2(\vec{x_1}) & \dots & \psi_N(\vec{x_1}) \\
        \psi_1(\vec{x_2}) & \psi_2(\vec{x_2}) & \dots & \psi_N(\vec{x_2}) \\
        \vdots & \vdots & \ddots & \vdots \\
        \psi_1(\vec{x_N}) & \psi_2(\vec{x_N}) & \dots & \psi_N(\vec{x_N}) \\
    \end{matrix} \right|,
\end{equation*}$$

  där $\vec{x}_i$ är rums- och spinnkoordinaterna för partikel $i$ \cite[s.~53]{Thijssen2007ComputationalPhysics}. Detta motiveras i sektion \ref{sec:wave_fn_electron_orbitals}. Dessa teoretiska partiklar benämns \emph{Kohn-Sham-partiklar}, och beter sig precis som vanliga elektroner med undantaget för växelverkan sinsemellan. För att Kohn-Sham-partiklarna ska bete sig likt elektroner införs en extra term i potentialen som approximativt kompenserar avsaknaden av växelverkan. 

 * **Uttryck problemet i termer av elektrontäthet**:

 Att uttrycka problemet i termer av elektrontätheten $n(\vec{r})$ förenklar problemet avsevärt, eftersom antalet 
 koordinater reduceras från tre per elektron till totalt tre. Alla potentialer och energier kan uttryckas som 
 funktional av elektrontätheten, och är enligt _Första Hohenberg-Kohn-teoremet_  ekvivalent med att uttrycka
 dem i termer av vågfunktioner.\cite[s.~8]{Finocchi2011DensityApproaches}. Hur elektrontätheten beräknas
 från elektronernas vågfunktioner specificeras i avsnitt \ref{sec:wave_fn_electron_orbitals}. 
 
[//]: # (Den sista meningen kan vi nog ta bort. Vi kommer nog gå genom det i kod sedan :)

Utifrån dessa förenklingar kan Schrödingerekvationen för varje Kohn-Sham-partikel skrivas om som ett egenvärdesproblem som kallas _Kohn-Shams ekvation_

$$\begin{equation}
    \label{eq:kohn}
    \left[-\frac{1}{2}\nabla^2 + V_\text{eff}(\vec{r}) \right] \psi_i(\vec{r}) = \varepsilon_i \psi_i(\vec{r}),
\end{equation}$$

där $\varepsilon_i$ är energiegenvärden och $V_\text{eff}$ den _effektiva potentialen_ som respektive Kohn-Sham-partikel upplever. Beräkningsalgoritmen, som kallas _Hartree–Fock-metoden_, bygger på att upprepade gånger lösa denna ekvation. Varje enskilt steg förklaras och motiveras i de nästkommande kapitlen. En visualisering av tillvägagångssättet visas i figuren nedan, och kan sammanfattas i 6 steg:

<img src="bilder/SCF_varm.svg" alt="Drawing" style="width:50%;"/>

1. Systemet diskretiseras i rummet och en inledande gissning av vågfunktionerna $\psi_i^{(0)}$ ställs upp.

2. I varje punkt beräknas elektrontätheten $n(\vec{r})^{(0)}$ (avsnitt \ref{sec:wave_fn_electron_orbitals})

3. Från den beräknade elektrontätheten $n(\vec{r})^{(0)}$ utvärderas den effektiva potentialen $V_\text{eff}$ 
   och systemets totala energi $E[n]$ (avsnitt \ref{sec:teori:potential}).

4. Kohn-Shams ekvation (\ref{eq:kohn}) löses med hjälp av $V_\text{eff}$, vilket ger nya elektronorbitaler 
   $\psi_i^{(1)}$ (avsnitt \ref{sec:teori:kohn}).

5. De nya elektronorbitalerna används i steg 2 för att hitta $n(\vec{r})^{(1)}$ och den tillhörande 
   energin $E[n]$, och processen upprepas.

6. Om skillnaden i total energi $E[n]$ mellan två iterationer är mindre än något bestämt 
   tröskelvärde stoppas programmet. Grundtillståndsenergin $E[n(\vec{r})^{(\text{final})}]$ 
   och grundtillståndselektronorbitalerna $\psi_i^{(\text{final})}$ är nu beräknade med 
   den önskade noggrannheten.

I denna notebook presenteras ett exempel på DFT implementerat i Python för att beräkna grundtillståndet för helium. Helium är ett flerelektronsystem som är sfäriskt symmetriskt, vilket förenklar implementationen. 

Hur hade det varit om vi presenterade länkar här till olika delsteg? Typ en sida som presenterar hur veff beräknas, en sida som presenterar total energi, en sida som visar hur slutliga implementationen fungerar. 